In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
dbListTables(con) %>% head

[1] "callout"          "chartevents_10"   "d_icd_procedures" "d_items"         
[5] "d_labitems"       "labevents"

In [5]:
cohort_sql <- readr::read_file(file.path(sql_dir, "cohort.sql"))
cohort_sql

[1] "with icu_age as (\n    select *,\n           date_part('year', age(intime, dob)) as age\n      from icustays\n      left join patients using (subject_id)\n)\n\n-- select intime, dob, age from icu_age limit 10;\n\n, icu_adult as (\n    select icustay_id\n      from icu_age\n     where age >= 18\n)\n\n-- select * from icu_adult limit 10;\n\n, icu_first as (\n    select first_value(icustay_id) over (partition by subject_id order by intime) as icustay_id\n      from icustays\n)\n\n-- select * from icu_first limit 10;\n\n, icu_flt_id as (\n    select subject_id, hadm_id, icustay_id\n      from icustays\n      join icu_adult using (icustay_id)\n      join icu_first using (icustay_id)\n)\n\n-- select * from icu_flt_id limit 10; -- 53362\n\n, population as (\n    select subject_id, hadm_id, icustay_id, angus\n      from icu_flt_id\n      left join angus_sepsis using (subject_id, hadm_id)\n)\n\n-- select * from population;\n-- select count(*) from population where angus = 1; -- 16125\n\n, echo_group as (\n    select distinct hadm_id,\n           first_value(chartdate) over (partition by hadm_id order by chartdate) as echo_time\n      from noteevents\n     where lower(category) like 'echo'\n       and lower(description) like 'report'\n)\n\n-- select * from echo_group limit 10;\n\n, cohort as (\n    select *\n      from population\n      left join echo_group using (hadm_id)\n)\n\nselect * from cohort;"

In [6]:
rm_semi_colon_end <- function(s) {
    if (endsWith(s, ";")) return(s %>% substr(1, nchar(.) - 1))
    return(s)
}

In [7]:
cohort_view_sql <- sprintf("
    drop materialized view if exists cohort cascade;
    create materialized view cohort as (
        %s
    );
", rm_semi_colon_end(cohort_sql))

In [8]:
dbSendQuery(con, cohort_view_sql)

<PostgreSQLResult>

In [9]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE